In [ ]:
import pandas as pd
import numpy as np
from fuzzywuzzy import process

pt_df = pd.read_csv("./data/pt_ratings.csv")
salaries_df = pd.read_csv("./data/salaries.csv")
rmp_df = pd.read_csv("./data/rmp_ratings.csv")


In [ ]:
# Drop professors w/o any ratings / reviews
pt_df = pt_df.dropna(subset=["average_rating", "reviews"])
pt_df = pt_df.drop(columns=["slug"])

# Join names to help name search in salaries
for i, row in salaries_df.iterrows():
    parts = row["employee"].split(", ")
    # assert len(parts) == 2

    salaries_df.at[i, "name"] = " ".join(parts[::-1])

# Join names to help name search in rmp
def combineNames(row):
    if pd.isnull(row["tMiddlename"]):
        return row["tFname"] + " " + row["tLname"]
    return row["tFname"] + " " + row["tMiddlename"] + " " + row["tLname"]

rmp_df["name"] = rmp_df.apply(lambda row: combineNames(row), axis=1)


In [ ]:
pt_df.head()

In [ ]:
rmp_df.head()

In [ ]:
salaries_df.info()

In [ ]:
pt_df["type"].value_counts()

In [ ]:
pt_df["average_rating"].describe()

In [ ]:
assert len(pt_df) == len(pt_df.drop_duplicates(subset=["slug"]))

In [ ]:
salaries_df["employee"] = salaries_df["employee"].str.replace("\n", " ")

In [ ]:
try:
    from cfuzzyset import cFuzzySet as FuzzySet
except ImportError:
    from fuzzyset import FuzzySet

salary_fuzz = FuzzySet()
for name in salaries_df["name"].unique():
    salary_fuzz.add(name)

pt_to_salary_name_map = {}
l =len(pt_df["name"].unique())

for i, name in enumerate(pt_df["name"].unique()):
    match = salary_fuzz.get(name)
    print(f"[{i + 1}/{l}] {name} -> {match}")
    pt_to_salary_name_map[name] = match

In [ ]:
# pd.DataFrame.from_dict(pt_to_salary_name_map)

In [ ]:
salary_names = salaries_df["name"].unique()

pt_to_salary_name_map = {}
l =len(pt_df["name"].unique())

for i, name in enumerate(pt_df["name"].unique()):
    # match = process.extract(name, salary_names)
    match = process.extractOne(name, salary_names, score_cutoff=90)
    print(f"[{i + 1}/{l}] {name} -> {match}")
    pt_to_salary_name_map[name] = match

    # if match:
    #     salaries = salaries_df[salaries_df["name"] == match[0]]
    #     assert len(salaries["name"].unique()) == 1
    #     display(salaries)

    #     pt_entries = pt_df[pt_df["name"] == name]
    #     display(pd.concat([pt_entries, salaries]))



In [ ]:
salary_names = salaries_df["name"].unique()

for name in np.random.choice(rmp_df["name"].unique(), 10):
    match = process.extract(name, salary_names)
    print(f"{name} -> {match}")

    # if match:
    #     salaries = salaries_df[salaries_df["name"] == match[0]]
    #     assert len(salaries["name"].unique()) == 1
    #     display(salaries)

    #     rmp_entries = rmp_df[rmp_df["name"] == name]
    #     display(pd.concat([rmp_entries, salaries]))

